<a href="https://colab.research.google.com/github/dguo4/algo_trading/blob/master/starter_files/001_equal_weight_S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [47]:
import numpy as np
import pandas as pd
import requests # API call 
import math
import time

In [12]:
!pip install xlsxwriter

import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [13]:
# read ticker 
stocks = pd.read_csv('sp_500_stocks.csv')
# stocks

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [21]:
# the import function doesn't work on my pc
# from secrets import IEX_CLOUD_API_TOKEN 
IEX_CLOUD_API_TOKEN = 'Tsk_c9ff8a39b60f46c8bd4e37d442d987ed'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [29]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' #f string
data = requests.get(api_url).json()
# print(data.status_code)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SNCSGOTKNEA)LE( B LLATERAMGS QDA/', 'calculationPrice': 'close', 'open': 131.53, 'openTime': 1660030700464, 'openSource': 'liffioca', 'close': 131.2, 'closeTime': 1622247013154, 'closeSource': 'lfiafcoi', 'high': 133.19, 'highTime': 1666927848895, 'highSource': 'crdemad uetn1pee 5l iyi', 'low': 131.04, 'lowTime': 1610863566522, 'lowSource': 'i  pear XlreeictIEm', 'latestPrice': 128.1, 'latestSource': 'Close', 'latestTime': 'January 6, 2021', 'latestUpdate': 1614846766703, 'latestVolume': 159635095, 'iexRealtimePrice': 128.36, 'iexRealtimeSize': 102, 'iexLastUpdated': 1664079890474, 'delayedPrice': 129.59, 'delayedPriceTime': 1663603216255, 'oddLotDelayedPrice': 127.5, 'oddLotDelayedPriceTime': 1624932653625, 'extendedPrice': 132.5, 'extendedChange': 1.1, 'extendedChangePercent': 0.00896, 'extendedPriceTime': 1641550618414, 'previousClose': 135.82, 'previousVolume': 100856730, 'change': -4.63, 'changePercent': -0.035, 'v

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [31]:
price = data['latestPrice']
market_cap = data['marketCap']
market_cap/1e12

2.1505685353

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [38]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame( columns = my_columns, )

In [40]:
final_dataframe.append(
    pd.Series(
        [symbol, price, market_cap/1e12, 'N/A'], 
        index=my_columns
    ), 
    ignore_index=True, 
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,128.1,2.150569,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [48]:
%%time

final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
  time.sleep(2)
  print(stock)
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}' #very slow to request using url
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
      pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A'],index=my_columns), 
      ignore_index=True
  )

A
AAL
AAP
AAPL
ABBV
CPU times: user 90.1 ms, sys: 4.35 ms, total: 94.4 ms
Wall time: 10.2 s


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: